In [808]:
import re
import math
from itertools import repeat, chain
from functools import partial, cached_property
import shapely

In [809]:
class Point(object):
    def __init__(self, point: tuple[int,int],round=12) -> None:
        self._point = point
        self._round = round

    @property
    def x(self):
        return round(self._point[0],self._round)

    @property
    def y(self):
        return round(self._point[1],self._round)

    def rotated(self,degrees, **kwargs):
        theta = degrees / 360 * 2 * math.pi
        xr = math.cos(theta) * self.x - math.sin(theta) * self.y
        yr = math.sin(theta) * self.x + math.cos(theta) * self.y
        return Point((xr,yr),**kwargs)
    
    def dist(self,point):
        return abs(self.x - point.x) + abs(self.y - point.y)

    def __repr__(self) -> str:
        return f"<Point: ({self.x},{self.y})>"
        

class Sensor(Point):

    def __init__(self, location: tuple, closest_beacon: tuple) -> None:
        self._beacon = Point(closest_beacon)
        super().__init__(location)
    
    @property
    def perimeter(self):
        return self.dist(self._beacon)

    @property
    def points(self):
        return (
            Point((self.x, self.y - self.perimeter)),
            Point((self.x+self.perimeter,self.y)),
            Point((self.x, self.y + self.perimeter)),
            Point((self.x - self.perimeter, self.y))
        )

    @property
    def polygon(self):
        return shapely.Polygon([(p.x,p.y) for p in self.points])
    
    @property
    def center(self):
        return Point((self.x,self.y))

    def rotated(self, degrees):
        return [point.rotated(degrees) for point in self.points]
        

    def __repr__(self) -> str:
        return f"<Sensor - ({self.x},{self.y}) - perimeter: {self.perimeter}>"


class Beacon(Point):
    def __init__(self, location: tuple, probing_sensor: Sensor) -> None:
        self._sensor = probing_sensor
        super().__init__(location)    

    def __repr__(self) -> str:
        return f"<Beacon - ({self.x},{self.y}) - scanned by: {self._sensor}>"


class Zone(object):

    def __init__(self, corners: list[Point]) -> None:
        self._corners = corners
        pass

    
    def __repr__(self) -> str:
        return f"<Zone defined by corners: {self._corners}"



class Map(object):

    def __init__(self,file: str,pad: bool = True) -> None:
        self._pad = pad
        self._file = file
        self._input = self._read_input()
        self._sensors: list[Sensor] = [Sensor(*x) for x in self._input]
        self._beacons: list[Beacon] = [Beacon(x[1],y) for x,y in zip(self._input,self._sensors)]
        pass

    def _read_input(self) -> list[list[tuple[int,int],tuple[int,int]]]:
        with open(self._file,'r') as file:
            input = [[tuple(map(int,re.findall(r"([-]*\d+)",y))) for y \
                 in i.split(':')] for i in file.read().splitlines()]
            return input

    @cached_property
    def objects(self):
        return self._sensors + self._beacons

    @cached_property
    def extent(self) -> tuple[tuple[int,int],tuple[int,int]]:
        xs = [point.x for point in self.objects]
        ys = [point.y for point in self.objects]

        if self._pad:
            mp = max([s.perimeter for s in self._sensors])
            return (min(xs)-mp,min(ys)-mp),(max(xs)+mp,max(ys)+mp)
        else:
            return (min(xs),min(ys)), (max(xs),max(ys))

            


In [810]:
m = Map('./assets/input_day_15.txt',pad=False)

In [827]:
fov = shapely.Polygon([(0,0),(4000000,0),(4000000,4000000),(0,4000000)])
union = shapely.union_all([x.polygon for x in m._sensors])
b = shapely.difference(fov,shapely.intersection(union,fov))

In [846]:
for x in set(b.exterior.coords.xy[0]):
    for y in set(b.exterior.coords.xy[1]):
        temp = Point((int(x),int(y)))
        if not any([temp.dist(sensor) <= sensor.perimeter for sensor in m._sensors]):
            print(x * 4000000 + y)


# [Point((x,y)) for x,y in zip(*b.exterior.coords.xy[:])]

12625383204261.0


In [259]:
y_row = 2000000
len(set(chain(*[_insight(inp,y_row) for inp in input]))) - len(set([x[1] for x in input if x[1][1]== y_row])) - len(set([x[0] for x in input if x[0][1]== y_row]))



5688618